In [1]:
import os
from dask_gateway import Gateway
from dask_gateway import GatewayCluster
from dask.distributed import Client


In [2]:
# Dask helpers
def slam(client,action,data,partition_factor=1.5):
    np = sum(client.nthreads().values())
    print('slam: np = %i'%np)
    shard_bounds = [int(i*len(data)/(1.0*partition_factor*np)) for i in range(int(partition_factor*np))] 
    if shard_bounds[-1] != len(data):
        print('a sb[-1]: ',shard_bounds[-1],len(data))
        shard_bounds = shard_bounds + [len(data)]
    print('sb: ',shard_bounds)
    data_shards = [data[shard_bounds[i]:shard_bounds[i+1]] for i in range(len(shard_bounds)-1)]
    print('ds len:        ',len(data_shards))
    print('ds item len:   ',len(data_shards[0]))
    print('ds type:       ',type(data_shards[0]))
    try:
        print('ds dtype:      ',data_shards[0].dtype)
    except:
        pass
    big_future = client.scatter(data_shards)
    results    = client.map(action,big_future)
    return results


In [3]:
gateway = Gateway(); gateway

Gateway<http://10.100.222.95:8000/services/dask-gateway>

In [4]:
options = gateway.cluster_options(); options

In [55]:
cluster = gateway.new_cluster(); cluster

In [56]:
clusters = gateway.list_clusters(); clusters

[ClusterReport<name=daskhub.bd2909b4c461495e98f1a0686fe6cbaa, status=RUNNING>]

In [57]:
client = cluster.get_client(); client

Client Scheduler: gateway://traefik-dhub-dask-gateway.daskhub:80/daskhub.bd2909b4c461495e98f1a0686fe6cbaa Dashboard: /services/dask-gateway/clusters/daskhub.bd2909b4c461495e98f1a0686fe6cbaa/status,Cluster Workers: 1 Cores: 2 Memory: 4.29 GB


In [24]:
# file_name='/home/jovyan/data/tmp/MOD05_L2.A2019337.1225.061.2019338011744.hdf'
# os.system('cd /home/jovyan/data/tmp; /home/jovyan/bin/createSidecarFile '+file_name)

0

In [ ]:
# TODO Teach createSidecarFile to understand S3 files.

In [63]:
import subprocess

def slam_action(inputs):
    ret=[]
    for vars in inputs:
        file_name = vars
        # ok = os.system('cd /home/jovyan/data/tmp; /home/jovyan/bin/createSidecarFile '+file_name) # nope
#        exe = subprocess.Popen(['/home/jovyan/bin/createSidecarFile',file_name]
#                               ,cwd='/home/jovyan/data/tmp'
#                               ,stdout=subprocess.PIPE
#                               ,stderr=subprocess.STDOUT
#                              )
        exe = subprocess.Popen(['ls','/home/jovyan']
#                               ,cwd='/home/jovyan/data/tmp/'
                               ,stdout=subprocess.PIPE
                               ,stderr=subprocess.STDOUT
                              )
        stdout,stderr=exe.communicate()
        # print('vars: ',file_name)
        ret.append("status fn=%s\nout=%s\nerr=%s"%(file_name,stdout,stderr))
    return ret

# result = slam(client,make_virtual_from,inputs,partition_factor=1)
# client.gather(result)

In [23]:
os.system('touch /tmp/pig')
!ls -l /tmp/pig; rm /tmp/pig

-rw-r--r-- 1 jovyan root 0 Oct 19 06:34 /tmp/pig


In [43]:
!pwd

/home/jovyan/git/STARE-Cookbooks/contrib/jupyter


In [59]:
import glob
basedir = '/home/jovyan/data/tmp/'
files = glob.glob(basedir+'MOD05_L2*.hdf')[0:1]
files

['/home/jovyan/data/tmp/MOD05_L2.A2019337.1225.061.2019338011744.hdf']

In [60]:
inputs=["a","b","c","d","e","f"]
inputs=inputs+inputs
inputs=["a","b","c"]
# inputs=["a","b"]
inputs = files

In [61]:
inputs

['/home/jovyan/data/tmp/MOD05_L2.A2019337.1225.061.2019338011744.hdf']

In [64]:
results = slam(client,slam_action,inputs)
cgresults = client.gather(results)

slam: np = 2
a sb[-1]:  0 1
sb:  [0, 0, 0, 1]
ds len:         3
ds item len:    0
ds type:        <class 'list'>


In [65]:
[i.result() for i in results]

[[],
 [],
 ["status fn=/home/jovyan/data/tmp/MOD05_L2.A2019337.1225.061.2019338011744.hdf\nout=b'Dockerfile\\nconda-linux-64.lock\\ndask-worker-space\\nenvironment.yml\\npostBuild\\nstart\\n'\nerr=None"]]

In [66]:
client.close()

In [67]:
cluster.shutdown()

In [68]:
cluster.close()

In [69]:
gateway.list_clusters()

[]

In [70]:
gateway.close()